# EDA Industry Part 1

Workflow: 0 

Goal: To create plots for Exploration page.

In [1]:
import os, sys, time, json
import pandas as PD
import numpy as NP

import plotly.graph_objects as GO
import plotly.figure_factory as FF
import plotly.subplots as SB

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'last'

In [3]:
os.chdir(os.getenv('PWD'))
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "settings.py")
import django
django.setup()

In [4]:
import movies.models.tables as TB
import movies.models.exploration as XT
import recommend.models.analysis as NL

## Movies Per Year

In [5]:
XT.Explore.GetYearPlot()

'{"data": [{"marker": {"color": "DarkOliveGreen"}, "x": ["2013", "2008", "2016", "2014", "1997", "2016", "2012", "1991", "2018", "2013", "2014", "2012", "2000", "2008", "2016", "2011", "2014", "2013", "2011", "2011", "2012", "2003", "1987", "2013", "1984", "2015", "2009", "2013", "1987", "2015", "2006", "2010", "2006", "2009", "2017", "2006", "2019", "2016", "2015", "2005", "2014", "2018", "2013", "2014", "2018", "1984", "2016", "2011", "2016", "2006", "2007", "2016", "2016", "2014", "2008", "2018", "2017", "1992", "2008", "2002", "2016", "2018", "2014", "2015", "2015", "2017", "2012", "2018", "2010", "2013", "2017", "2017", "2011", "2015", "2015", "2012", "2004", "2014", "2013", "2017", "2013", "2012", "2012", "2011", "2011", "2011", "2012", "2012", "2015", "2016", "2013", "2018", "2008", "2014", "2016", "2017", "2002", "2016", "2015", "1992", "2010", "2018", "2016", "2016", "2016", "2019", "2013", "2012", "2006", "2012", "2013", "2014", "2009", "2011", "2012", "2016", "2006", "2008",

In [6]:
master _ls = list(MD.MasterMovie.objects.values())
master_df = PD.DataFrame(master_ls)
master_df = master_df.drop(columns=['id', 'OriginalTitle', 'Poster', 'Synopsis', 'Indeces'])
master_df.info()

SyntaxError: invalid syntax (<ipython-input-6-e632f261f973>, line 1)

In [ ]:
master_df = master_df.loc[master_df['Year']>='1980']
master_df.shape

In [ ]:
fig = GO.Figure()
fig.add_trace(GO.Histogram(x=master_df['Year'], xbins={'size': 1}, marker_color='DarkOliveGreen'))

fig.update_layout(
    title="Number of Movies Per Year",
    xaxis_title="Year",
    yaxis_title="Movies Count",
    width=700,
    height=500,
    margin=GO.layout.Margin(t=50, r=20, b=50, l=80, pad=0),
    paper_bgcolor="LightSteelBlue",
)
#fig.update_xaxes(categoryorder="total descending")

## Movie Countries

In [ ]:
XT.Explore.GetCountriesPlot()

In [ ]:
master_ls = list(MD.MasterMovie.objects.values())
master_df = PD.DataFrame(master_ls)
master_df = master_df.drop(columns=['id', 'OriginalTitle', 'Poster', 'Synopsis', 'Indeces'])
master_df.info()

In [ ]:
# drop nan's 

master_df = master_df.dropna(axis=0, subset=['Country'])
master_df = master_df.loc[master_df['Country']!='nan'].reset_index().copy()
master_df.info()

In [ ]:
group_df = master_df.groupby('Country').count()['Movie_ID'].sort_values(ascending=False)
group_df.head(10)

In [ ]:
others_df = group_df.loc[group_df.index!='USA']
others_df.head()

In [ ]:
primary_x = ['USA', 'All Others']
primary_y = [group_df[group_df.index=='USA'].values[0], others_df.sum()]
primary_y

In [ ]:
plot_df = others_df[:12]

In [ ]:
COLOR = 'mediumpurple'
fig = SB.make_subplots(rows=1, cols=2, column_widths=[0.25, 0.75], 
                       subplot_titles=("Top Countries", "Secondary Countries"))

fig.add_trace(
    GO.Bar(x=primary_x, y=primary_y, marker_color=COLOR, hoverlabel={'namelength': 0}), 
    row=1, col=1)
fig.add_trace(
    GO.Bar(x=plot_df.index, y=plot_df.values, marker_color=COLOR, hoverlabel={'namelength': 0}), 
    row=1, col=2)

fig.update_layout(
    title={'text': "Countries That Make Movies", 'x':0.5, 'y':0.99,  
        'xanchor': 'center', 'yanchor': 'top'},
    width=700,
    height=500,
    margin=GO.layout.Margin(t=50, r=20, b=50, l=70, pad=0),
    paper_bgcolor="LightSteelBlue",
    showlegend=False,
)

# extend annotations so they don't override the subplot titles

annotations = [a.to_plotly_json() for a in fig["layout"]["annotations"]]
#annotations.append(dict(x=0.5, y=-0.06, text="User Votes"))
# annotations.append({'font': {'size': 16}, 'showarrow': False, 'text': 'User Votes', 
#                     'x': 0.5, 'xanchor': 'center', 'xref': 'paper', 
#                     'y': -0.06, 'yanchor': 'bottom', 'yref': 'paper'})
#annotations.append(dict(x=-0.05, y=0.4, text="Movies Count", textangle=-90))
annotations.append({'font': {'size': 16}, 'showarrow': False, 'text': 'Movie Counts', 
                    'x': -0.08, 'xanchor': 'center', 'xref': 'paper', 
                    'y': 0.4, 'yanchor': 'bottom', 'yref': 'paper', 'textangle':-90})
fig["layout"]["annotations"] = annotations

# fig.update_xaxes(tickvals=vote_classes, range=[0.4, 3.6])
# fig.update_yaxes(range=[0, 17000])

fig.show()

In [ ]:
fig = GO.Figure()
fig.add_trace(
    GO.Bar(x=plot_df.index, y=plot_df.values, marker_color='mediumpurple') )

fig.update_layout(
    title="Countries That Make Movies",
    xaxis_title="",
    yaxis_title="Movie Count",
    width=600,
    height=500,
    margin=GO.layout.Margin(t=50, r=20, b=10, l=70, pad=0),
    paper_bgcolor="LightSteelBlue",
)
# fig.update_xaxes(tickvals=vote_xy[0])
# fig.update_yaxes(tickvals=list(range(0, 1000, 100)))